In [2]:
from yelpapi import YelpAPI

In [40]:
import requests 
import json
import pandas as pd
from pandas.io.json import json_normalize
from pprint import pprint

file = open('YelpAPI_Data.json', 'w+')
api_key="ne_rTJNNgC4JzUhnL2hnia9eYPphCe-G5JpNILy2a5pH6jN4D3ZOFTOke2SXS28MaZTCtNrfs_kBWWg6eKl_hu1TlCaeixccKJywA1JmH-OgQpQRYGhwYYdyfwZXXnYx"  #put your key
url="https://api.yelp.com/v3/businesses/search"
cities = 'New York, NY'
dict_list=[]

params={'location':cities,'limit':50}
headers= {'Authorization': 'bearer %s' % api_key}
response=requests.get(url, headers=headers, params=params)
    
    # proceed only if the status code is 200
print('The status code is {}'.format(response.status_code))
    
d1=response.json()
dict_list.append(d1)

The status code is 200


In [41]:
from pprint import pprint
import re
s=['coordinates','location']

#customized header
header =['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'category alias', 'category title', 'rating', 
         'coordinates', 'latitude', 'longitude', 
         'street', 'apt', 'area','city','pincode','country','state','address1','address2',
         'phone', 'display_phone', 'distance', 'price']

csv_data = []
csv_data.append(header)
for city in dict_list:
    for n, item in enumerate(city):
        if item == 'businesses':
            for value in city[item]:
                row = []
                price = ' '
                for each in value:
                    if each in s:
                        for key in value[each]:
                            if type(value[each][key]) is list:
                                 row.append("-".join(value[each][key]))
                            else:
                                row.append(value[each][key])
                    elif each=='categories':
                        for j in value[each][0]:
                            row.append(value[each][0][j])
                    elif each=='transactions':
                        row.append("-".join(value[each]))
                    elif each=='price':
                        price = value[each]
                    else:
                        val = re.sub(r'[^\x00-\x7F]+','', str(value[each]))
                        row.append('"' + val + '"')

                row.append(price)
                #print(row)
                csv_data.append(row)

In [42]:
import csv
from csv import DictWriter
with open('yelp1.csv', 'w', encoding='utf8') as f:
    for row in csv_data:
        f.write(",".join([str(val) for val in row]) + "\n")

In [43]:
df=pd.read_csv('yelp1.csv')

In [44]:
df.shape

(50, 26)

In [46]:
df.tail()

,id,alias,name,image_url,is_closed,url,review_count,category alias,category title,rating,...,city,pincode,country,state,address1,address2,phone,display_phone,distance,price
45,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2197,italian,Italian,4.5,...,New York,10012,US,NY,235 Mulberry St-New York,NY 10012,1.212965e+10,(212) 965-0500,1931.268372,$$
46,-OixbLnFLCzQclxCSbUQ8w,up-thai-new-york,Up Thai,https://s3-media3.fl.yelpcdn.com/bphoto/g9J3na...,False,https://www.yelp.com/biz/up-thai-new-york?adju...,2236,thai,Thai,4.5,...,New York,10021,US,NY,1411 2nd Ave-New York,NY 10021,1.212256e+10,(212) 256-1188,7796.912676,$$
47,HxC2ZN010NxAFwjTqUVpzw,rice-to-riches-new-york,Rice To Riches,https://s3-media3.fl.yelpcdn.com/bphoto/YcJMOV...,False,https://www.yelp.com/biz/rice-to-riches-new-yo...,3004,desserts,Desserts,4.0,...,New York,10012,US,NY,37 Spring St-New York,NY 10012,1.212274e+10,(212) 274-0008,1828.038283,$$
48,dMhRafXdr765DHe0k-QfaQ,abc-kitchen-new-york,ABC Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/k2V0Qy...,False,https://www.yelp.com/biz/abc-kitchen-new-york?...,2966,newamerican,American (New),4.0,...,New York,10003,US,NY,35 E 18th St-New York,NY 10003,1.212476e+10,(212) 475-5829,3652.084379,$$$
49,iBm8YTqNwrddsxWdqLPK-A,caracas-arepa-bar-new-york,Caracas Arepa Bar,https://s3-media3.fl.yelpcdn.com/bphoto/wtb4bA...,False,https://www.yelp.com/biz/caracas-arepa-bar-new...,2551,venezuelan,Venezuelan,4.0,...,New York,10009,US,NY,91 E 7th St-New York,NY 10009,1.212229e+10,(212) 228-5062,2507.056913,$$
